In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

/home/vincent/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/vincent/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


- Phase finale de nettoyage (gestion des NaN et remplacement des catégories par des nombres).
- Entraîner des modèles 
- Idéntifier les bonnes affaires

# Récupération des données

In [2]:
data = pd.read_csv("../donnees/data.tsv", sep="\t")

In [3]:
data.head()

,Id,Genre,Neuf,Surface,Pieces,Quartier,Prix
0,annonce-138905473-376235,Appartement,0,90.00,3.0,cathédrale,374400.0
1,annonce-140620177-376235,Appartement,0,146.27,5.0,sud,499200.0
2,annonce-140620179-376235,Appartement,0,110.00,5.0,prébendes,499200.0
3,annonce-133494153-376235,Maison,0,132.00,6.0,prébendes,508000.0
4,annonce-137425993-376235,Maison,0,185.00,7.0,strasbourg,676000.0


In [4]:
print(data.dtypes)

Id           object
Genre        object
Neuf          int64
Surface     float64
Pieces      float64
Quartier     object
Prix        float64
dtype: object


# Gestion Id

In [5]:
data.drop("Id", axis=1, inplace=True)

In [6]:
data.dtypes

Genre        object
Neuf          int64
Surface     float64
Pieces      float64
Quartier     object
Prix        float64
dtype: object

# Gestion genre

In [7]:
genre = data.Genre.copy()

In [8]:
genre.isna().any()

False

**Pas de données manquantes. Juste une transformation en numérique à effectuer.**

In [9]:
apparts = (genre == "Appartement")

In [10]:
genre[apparts] = 0
genre[~apparts] = 1

In [11]:
data["Genre"] = genre.astype(np.int)

In [12]:
data.dtypes

Genre         int64
Neuf          int64
Surface     float64
Pieces      float64
Quartier     object
Prix        float64
dtype: object

# Gestion neuf

In [13]:
neuf = data.Neuf.copy()

In [14]:
neuf.isna().any()

False

**Pas de données manquantes, rien à faire**

# Gestion surface

In [15]:
surf = data.Surface.copy()

In [16]:
surf.isna().any()

True

**Données manquantes. On va insérer la moyenne.**

In [17]:
ind_valides = ~surf.isna()

In [18]:
moyenne = surf[ind_valides].mean()

In [19]:
surf[~ind_valides] = moyenne

In [20]:
surf.isna().any()

False

In [21]:
data["Surface"] = surf

# Gestion des pièces

In [22]:
pieces = data.Pieces.copy()

In [23]:
pieces.isna().any()

True

In [24]:
pieces.isna().sum()

1

**Une données manquantes. On va insérer la médiane pour rester avec des entiers.**

In [25]:
ind_valides = ~pieces.isna()

In [26]:
mediane = pieces[ind_valides].median()
mediane

3.0

In [27]:
pieces[~ind_valides] = mediane

In [28]:
pieces.isna().any()

False

In [29]:
data["Pieces"] = pieces.astype(np.int)

# Gestion des quartiers

In [30]:
quartiers = data[["Quartier"]].copy()

In [31]:
type(quartiers)

pandas.core.frame.DataFrame

In [32]:
quartiers.Quartier.isna().any()

True

**On des valeurs catégorielles et des valeurs manquantes. On va utiliser un onehotencoder.**

In [33]:
ind_valides = ~quartiers.Quartier.isna()

**Récupération des quartiers possibles.**

In [35]:
qs = np.unique(quartiers.Quartier[ind_valides].values)
qs

array(['2 lions', 'beaujardin', 'béranger', 'cathédrale', 'centre',
       'cher', 'cluzel', 'conservatoire', 'eloi', 'febvotte', 'fontaines',
       'gare', 'halles', 'heurteloup', 'mairie', 'montjoyeux', 'nord',
       'paul bert', 'portes', 'prébendes', 'radegonde', 'rotonde',
       'strasbourg', 'sud', 'tranchée', 'velpeau'], dtype=object)

In [37]:
for q in qs:
    nom = "Quartier={}".format(q)
    quartiers[nom] = 0
    quartiers.loc[quartiers.Quartier == q, nom] = 1

In [38]:
quartiers.head(30)

,Quartier,Quartier=2 lions,Quartier=beaujardin,Quartier=béranger,Quartier=cathédrale,Quartier=centre,Quartier=cher,Quartier=cluzel,Quartier=conservatoire,Quartier=eloi,...,Quartier=nord,Quartier=paul bert,Quartier=portes,Quartier=prébendes,Quartier=radegonde,Quartier=rotonde,Quartier=strasbourg,Quartier=sud,Quartier=tranchée,Quartier=velpeau
0,cathédrale,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,sud,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,prébendes,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,prébendes,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,strasbourg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
5,portes,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
6,mairie,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,prébendes,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
9,cathédrale,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
quartiers.drop("Quartier", axis=1, inplace=True)

In [42]:
quartiers.loc[~ind_valides] = np.NaN

In [43]:
quartiers.head(30)

,Quartier=2 lions,Quartier=beaujardin,Quartier=béranger,Quartier=cathédrale,Quartier=centre,Quartier=cher,Quartier=cluzel,Quartier=conservatoire,Quartier=eloi,Quartier=febvotte,...,Quartier=nord,Quartier=paul bert,Quartier=portes,Quartier=prébendes,Quartier=radegonde,Quartier=rotonde,Quartier=strasbourg,Quartier=sud,Quartier=tranchée,Quartier=velpeau
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
moyennes = quartiers[ind_valides].mean()
moyennes

Quartier=2 lions          0.019895
Quartier=beaujardin       0.029319
Quartier=béranger         0.002094
Quartier=cathédrale       0.036649
Quartier=centre           0.170681
Quartier=cher             0.094241
Quartier=cluzel           0.003141
Quartier=conservatoire    0.004188
Quartier=eloi             0.005236
Quartier=febvotte         0.015707
Quartier=fontaines        0.003141
Quartier=gare             0.054450
Quartier=halles           0.067016
Quartier=heurteloup       0.009424
Quartier=mairie           0.005236
Quartier=montjoyeux       0.012565
Quartier=nord             0.193717
Quartier=paul bert        0.010471
Quartier=portes           0.019895
Quartier=prébendes        0.063874
Quartier=radegonde        0.018848
Quartier=rotonde          0.001047
Quartier=strasbourg       0.014660
Quartier=sud              0.094241
Quartier=tranchée         0.012565
Quartier=velpeau          0.037696
dtype: float64

In [54]:
quartiers.loc[~ind_valides, :] = moyennes.values.reshape((1, moyennes.values.size))

In [56]:
quartiers.head(20)

,Quartier=2 lions,Quartier=beaujardin,Quartier=béranger,Quartier=cathédrale,Quartier=centre,Quartier=cher,Quartier=cluzel,Quartier=conservatoire,Quartier=eloi,Quartier=febvotte,...,Quartier=nord,Quartier=paul bert,Quartier=portes,Quartier=prébendes,Quartier=radegonde,Quartier=rotonde,Quartier=strasbourg,Quartier=sud,Quartier=tranchée,Quartier=velpeau
0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,1.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
7,0.019895,0.029319,0.002094,0.036649,0.170681,0.094241,0.003141,0.004188,0.005236,0.015707,...,0.193717,0.010471,0.019895,0.063874,0.018848,0.001047,0.01466,0.094241,0.012565,0.037696
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000


In [63]:
data = pd.concat([data.drop("Quartier", axis=1), quartiers], axis=1)

# Gestion prix

In [65]:
prix = data.Prix.copy()

In [66]:
prix.isna().any()

True

In [67]:
prix.isna().sum()

1

**Une seule donnée manquante, on va l'enlever.**

In [68]:
data.dropna(inplace=True)

In [70]:
data.isna().any()

Genre                     False
Neuf                      False
Surface                   False
Pieces                    False
Prix                      False
Quartier=2 lions          False
Quartier=beaujardin       False
Quartier=béranger         False
Quartier=cathédrale       False
Quartier=centre           False
Quartier=cher             False
Quartier=cluzel           False
Quartier=conservatoire    False
Quartier=eloi             False
Quartier=febvotte         False
Quartier=fontaines        False
Quartier=gare             False
Quartier=halles           False
Quartier=heurteloup       False
Quartier=mairie           False
Quartier=montjoyeux       False
Quartier=nord             False
Quartier=paul bert        False
Quartier=portes           False
Quartier=prébendes        False
Quartier=radegonde        False
Quartier=rotonde          False
Quartier=strasbourg       False
Quartier=sud              False
Quartier=tranchée         False
Quartier=velpeau          False
dtype: b

In [71]:
data.to_parquet(fname="../donnees/propre.parquet")